<div style="background-color:#783465 ; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white"> Evaluatie Methoden </h1>
    
<h4 style="color:black"> Datalab V: Julia Boschman, Isa Dijkstra, Daan Eising, Rogier Gernaat </h4>

<a name='start'></a>

## Table of contents

1. [H1: Inleiding](#1.0)
    - [&sect;1.1: Uitleg van het Notebook](#1.1)
    - [&sect;1.2: Imports](#1.2)
2. [H2: Inladen gegenereerde bestanden](#2.0)
3. [H3: Evaluatie Technieken](#3.0)
    - [&sect;3.1: Structuuranalyse](#3.1)
    - [&sect;3.2: Volledigheidscheck](#3.2)
    - [&sect;3.3: Bruikbaarheid](#3.3)
    - [&sect;3.4: Objectieve kwaliteitsmetingen](#3.4)
    - [&sect;3.5: Ethische aspecten](#3.5)
4. [H4: Resultaten](#4.0)
    - [&sect;4.1: Structuuranalyse](#4.1)
    - [&sect;4.2: Volledigheidscheck](#4.2)
        - [&sect;4.2.1: Volledigheid Penta Jacob van Liesveldt College](#4.2.1)
        - [&sect;4.2.2: Volledigheid Johannes Calvijnschool](#4.2.2)
        - [&sect;4.2.3: Volledigheid Liemers College](#4.2.3)
        - [&sect;4.2.4: Volledigheid Penta Jacob van Liesveldt College OpenAI API](#4.2.4)
    - [&sect;4.3: Bruikbaarheid](#4.3)
    - [&sect;4.4: Objectieve kwaliteitsmetingen](#4.4)
    - [&sect;4.5: Ethische aspecten](#4.5)
5. [H5: Conclusie](#5.0)
    - [&sect;5.1: Custom GPT](#5.1)
    - [&sect;5.2: OpenAI API](#5.2)
    - [&sect;5.3: Vergelijking](#5.3)
6. [H6: Bijlagen](#6.0)
    - [&sect;6.1: Checklist Structuuranalyse](#6.1)
    - [&sect;6.2: Vragenlijst invulling](#6.2)
    - [&sect;6.3: Checklist Ethische aspecten](#6.3)
7. [Literatuurlijst](#7.0)

[Back to the top](#start)
<a name='1.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H1: Inleiding</strong></h2>
</div>

<a name='1.1'></a>
<h3>&sect;1.1: Uitleg van het Notebook </h3>

In dit notebook worden verschillende concept-leermiddelenbeleidsplannen geëvalueerd de zijn gegenereerd met behulp van zowel de custom GPT-aanpak als de OpenAI API-aanpak. De evaluatie bestaat uit een reeks automatische checks op het gebied van structuur, volledigheid, objectieve tekstkwaliteit en ethische criteria. 

Daarnaast is er ook aandacht voor de bruikbaarheid van de gegenereerde plannen: enkele teksten zijn door onze opdrachtgever beoordeeld, waarbij feedback op het Penta Jacob van Liesveldt College is verwerkt in de versie voor de Johannes Calvijnschool in Urk. Door deze verschillende aspecten per document en per aanpak te vergelijken, ontstaat er een duidelijk beeld van de sterkte en zwakke punten van elk gegenereerd beleidsstuk. 

Het notebook biedt zo inzicht in de prestaties van de gebruikte AI-tools bij het genereren van leermiddelenbeleidsplannen voor scholen, zelfs wanneer het team weinig verstand heeft van de inhoud van een leermiddelenbeleid.

<a name='1.2'></a>
<h3>&sect;1.2: Imports </h3>

In [1]:
import os
from pdf2image import convert_from_path
import pytesseract
import re
import pandas as pd
from nltk.tokenize import word_tokenize
import textstat

C:\Users\bosch\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\bosch\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[Back to the top](#start)
<a name='2.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H2: Inladen gegenereerde bestanden</strong></h2>
</div>

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
poppler_path = r'C:\Program Files\poppler-24.08.0\Library\bin'

pdf_dir = 'Testen Bot'

In [3]:
def ocr_text_from_single_pdfs(pdf_dir):
    all_texts = {}
    for filename in os.listdir(pdf_dir):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, filename)
            school = os.path.splitext(filename)[0]
            print(f"OCR: {school} uit {pdf_path}")

            images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
            
            full_text = ""
            for img in images:
                text = pytesseract.image_to_string(img, lang='nld')
                full_text += text + "\n"
            
            all_texts[school] = full_text.strip()
            
    return all_texts

In [4]:
school_sections = ocr_text_from_single_pdfs(pdf_dir)

OCR: (OpenAI API) Concept LMB Jacob van liesveldt uit Testen Bot\(OpenAI API) Concept LMB Jacob van liesveldt.pdf
OCR: Concept Leermiddelenbeleid Johannes Calvijnschool uit Testen Bot\Concept Leermiddelenbeleid Johannes Calvijnschool.pdf
OCR: Concept Leermiddelenbeleid Liemers College uit Testen Bot\Concept Leermiddelenbeleid Liemers College.pdf
OCR: Concept Leermiddelenbeleid Penta Jacob van Liesveldt uit Testen Bot\Concept Leermiddelenbeleid Penta Jacob van Liesveldt.pdf


In [5]:
for school, text in school_sections.items():
    print(f"\n--- {school} ---\n")
    print(text)


--- (OpenAI API) Concept LMB Jacob van liesveldt ---

Concept Leermiddelenbeleid Penta Jacob van Liesveldt

(conceptversie — ter ondersteuning bij het opstellen van definitief beleid)

24/04/2025

1van17

Inhoudsopgave

1. Aanleiding, reikwijdte en doel...aseses sss sss e e 3
1.1 Aanleiding..... aa 3
1.2Reikwijdte …... esese sss e 4
1.3DOel n enee e e e e eeeeeeeeeeeeee 4

2. Onderwijskundige visie en doelstellingen…. sss sss sss sss 5
2.1 Kernwaarden en onderwijsdoelen …... esesesese sss 6
2.2 Ondersteuning door leermiddelen …... 7

3. Randvoorwaarden (ICT, financiën, HR) … 8
KC N O 8
3.2 FinanciëN.enee e e 9
3.3 HR- Deskundigheid en ondersteuning … ss sss sss 10

4. Richtlijnen voor keuze en aanschaf.…. sss 11
4.1 Selectiekaderg .….m eeneeeee e n ee e e 11
4.2 Aanschafproces...sssse sss sss sss 12

5. Leermiddelenmix en praktijkgebruik …… se sss sss 13
5.1 LeermiddelenMix …… awenen sss e e 13
5.2 Praktijkgebruik ….. wsssseesee sss sss e 14

6. Evaluatie en onderhoud van beleid …… en

[Back to the top](#start)
<a name='3.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H3: Evaluatie Technieken</strong></h2>
</div>

Voor dit hoofstuk is gebruik gemaakt van het document ["Evaluatietechnieken"](https://dehaagsehogeschool-my.sharepoint.com/:w:/r/personal/22162038_student_hhs_nl/_layouts/15/Doc.aspx?sourcedoc=%7B9F01B2FA-2E69-4F9D-9FFE-00FB1BBBE2D2%7D&file=Evaluatietechnieken.docx&action=default&mobileredirect=true) die terug te vinden is in onze Onedrive.

<a name='3.1'></a>
<h3>&sect;3.1: Structuuranalyse </h3>

Om een structuuranalyse te doen maken we een checklist op basis van de aanbevolen indeling van een leermiddelenbeleid. Deze lijst wordt opgebouwd door middel van het template van een leermiddelenbeleid, verkregen vanuit Kennisnet. 

Aanwezige hoofdstukken en paragrafen:
- [ ] Aanleiding, reikwijdte en doel
    - [ ] Aanleiding
    - [ ] Reikwijdte
    - [ ] Doel
- [ ] Onderwijskundige visie en doel
    - [ ] Kernwaarden en onderwijsdoelen
    - [ ] Ondersteuning door leermiddelen
- [ ] Randvoorwaarden
    - [ ] ICT
    - [ ] Financiën
    - [ ] HR
- [ ] Richtlijnen voor keuze en aanschaf leermiddelen
    - [ ] Selectiekaders
    - [ ] Aanschafproces
- [ ] Leermiddelenmix en praktijkgebruik
    - [ ] Leermiddelenmix
    - [ ] Praktijkgebruik
- [ ] Evaluatie en onderhoud van beleid
    - [ ] Gebruik
    - [ ] Frequenie en proces

Dee checklist is gebasseerd op een bestand, geleverd van Kennisnet, die momenteel wordt gebruikt als een invul format. Deze is terug te vinden in onze [OneDrive](https://dehaagsehogeschool-my.sharepoint.com/:w:/r/personal/22162038_student_hhs_nl/_layouts/15/Doc.aspx?sourcedoc=%7B41B278AA-D357-4FCA-A0B0-C3BC94DEDCDC%7D&file=Format%20vragen.docx&action=default&mobileredirect=true)

<a name='3.2'></a>
<h3>&sect;3.2: Volledigheidscheck </h3>

Hier beoordelen we of de gegenereerde tekst elk hoofdstuk en elke paragraaf ten minste kort bespreekt. Er mag niks belangrijks ontbreken. Zo kijken wij met scores per hoofdstuk en per paragraaf of alle vragen duidelijk zijn beantwoord. Hier kunnen wij bijvoorbeeld de scores zo verdelen: 
- 0 = geen enkele vraag beantwoord 
- 1 = een deel van de vragen beantwoord 
- 2 = (bijna) alle vragen beantwoord 
- 3 = alle vragen beantwoord 

De vragenlijst die wij hiervoor gebruiken is terug te vinden in hoofdstuk 6, [Bijlagen](#6.0)
<a name='3.0'></a>

<a name='3.3'></a>
<h3>&sect;3.3: Bruikbaarheid </h3>

Het is belangrijk dat niet iedere generatie van tekst hetzelfde is, over alle scholen. Dat zou betekenen dat een leermiddelenbeleid echt moet aansluiten bij de normen en waarden van de school en niet generiek moet blijven. Daarom kijken we ook naar de verschillen tussen teksten van andere soorten scholen. 

<a name='3.4'></a>
<h3>&sect;3.4: Objectieve kwaliteitsmetingen </h3>

Voor de analyse van de data hebben wij enkele bestaande leermiddelenbeleidsplannen bestudeerd. Om de door ons gegenereerde leermiddelenbeleidsplannen te beoordelen, zullen wij gebruikmaken van dezelfde statistieken, waarbij we vooral vergelijken met een eerder goedgekeurd voorbeeldbeleid. Wij kijken hierbij naar de gemiddelde zinlengte, de woorddiversiteit en het totaal aantal woorden per tekst. Daarnaast voegen we een leesbaarheidsanalyse toe, onder andere met de Flesch Reading Ease-score, om de moeilijkheidsgraad van de teksten te bepalen. Deze analyses voeren we zowel uit op bestaande als op door AI gegenereerde beleidsplannen, zodat we kunnen beoordelen in hoeverre de moeilijkheidsgraad en leesbaarheid van beide groepen overeenkomen.  

<a name='3.5'></a>
<h3>&sect;3.5: Ethische aspecten </h3>

Naast de inhoudelijke evaluatie moet er ook duidelijk gekeken worden naar de ethische aspecten van onze toepassingen. Zo kijken we naar de transparantie van de aanpakken. Het moet voor de gebruiker duidelijk zijn hoe het model bij de uitkomst is gekomen. Daarnaast is het belangrijk dat er ook gekeken moet worden naar inclusiviteit. Alle belangrijke eenheden moeten worden meegenomen in het leermiddelenbeleid. Denk hierbij aan de leerlingen, docenten, ouders en mogelijk andere betrokkenen.  De checklist die wij hiervoor gebruiken is terug te vinden in hoofdstuk 6, [Bijlagen](#6.0)
<a name='3.0'></a>

[Back to the top](#start)
<a name='4.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H4: Resultaten</strong></h2>
</div>

<a name='4.1'></a>
<h3>&sect;4.1: Structuuranalyse </h3>

In [6]:
structure_checklist = {
    "Aanleiding, reikwijdte en doel": [
        "Aanleiding",
        "Reikwijdte",
        "Doel"
    ],
    "Onderwijskundige visie en doel": [
        "Kernwaarden en onderwijsdoelen",
        "Ondersteuning door leermiddelen"
    ],
    "Randvoorwaarden": [
        "ICT",
        "Financiën",
        "HR"
    ],
    "Richtlijnen voor keuze en aanschaf": [
        "Selectiekaders",
        "Aanschafproces"
    ],
    "Leermiddelenmix en praktijkgebruik": [
        "Leermiddelenmix",
        "Praktijkgebruik"
    ],
    "Evaluatie en onderhoud van beleid": [
        "Gebruik",
        "Frequentie en proces"
    ]
}

In [7]:
def structuur_analyse(beleidstekst, checklist):
    beleidstekst_lower = beleidstekst.lower()
    result = {}
    totaal_hoofdstukken = len(checklist)
    totaal_paragrafen = sum(len(p) for p in checklist.values())
    gevonden_hoofdstukken = 0
    gevonden_paragrafen = 0

    checklist_per_hoofdstuk = {}

    for hoofdstuk, paragrafen in checklist.items():
        hoofdstuk_aanwezig = hoofdstuk.lower() in beleidstekst_lower
        if hoofdstuk_aanwezig:
            gevonden_hoofdstukken += 1
        paragraaf_dict = {}
        gevonden_paragrafen_h = 0
        for para in paragrafen:
            para_aanwezig = para.lower() in beleidstekst_lower
            if para_aanwezig:
                gevonden_paragrafen += 1
                gevonden_paragrafen_h += 1
            paragraaf_dict[para] = para_aanwezig
        checklist_per_hoofdstuk[hoofdstuk] = {
            'hoofdstuk_aanwezig': hoofdstuk_aanwezig,
            'paragrafen': paragraaf_dict,
            'paragraafscore': (gevonden_paragrafen_h / len(paragrafen) * 100) if paragrafen else None
        }
    hoofdstukscore = gevonden_hoofdstukken / totaal_hoofdstukken * 100
    paragraafscore = gevonden_paragrafen / totaal_paragrafen * 100 if totaal_paragrafen else None
    return {
        "hoofdstukscore %": round(hoofdstukscore, 1),
        "paragraafscore %": round(paragraafscore, 1) if paragraafscore is not None else None,
        "per_hoofdstuk": checklist_per_hoofdstuk
    }

In [11]:
resultaten = {}
for school, beleidstekst in school_sections.items():
    resultaten[school] = structuur_analyse(beleidstekst, structure_checklist)

In [12]:
for school, result in resultaten.items():
    print(f"\n==== {school} ====")
    print(f"Hoofdstukscore: {result['hoofdstukscore %']}%")
    print(f"Paragraafscore: {result['paragraafscore %']}%")
    
    ontbrekende_hoofdstukken = []
    ontbrekende_paragrafen = []
    
    for hoofdstuk, details in result["per_hoofdstuk"].items():
        if not details['hoofdstuk_aanwezig']:
            ontbrekende_hoofdstukken.append(hoofdstuk)
        for para, aanwezig in details['paragrafen'].items():
            if not aanwezig:
                ontbrekende_paragrafen.append(f"{hoofdstuk} > {para}")
    
    if ontbrekende_hoofdstukken:
        print("Ontbrekende hoofdstukken:")
        for h in ontbrekende_hoofdstukken:
            print(f"  - {h}")
    else:
        print("Alle hoofdstukken aanwezig!")
        
    if ontbrekende_paragrafen:
        print("Ontbrekende paragrafen:")
        for p in ontbrekende_paragrafen:
            print(f"  - {p}")
    else:
        print("Alle paragrafen aanwezig!")


==== (OpenAI API) Concept LMB Jacob van liesveldt ====
Hoofdstukscore: 100.0%
Paragraafscore: 100.0%
Alle hoofdstukken aanwezig!
Alle paragrafen aanwezig!

==== Concept Leermiddelenbeleid Johannes Calvijnschool ====
Hoofdstukscore: 100.0%
Paragraafscore: 92.9%
Alle hoofdstukken aanwezig!
Ontbrekende paragrafen:
  - Evaluatie en onderhoud van beleid > Frequentie en proces

==== Concept Leermiddelenbeleid Liemers College ====
Hoofdstukscore: 66.7%
Paragraafscore: 78.6%
Ontbrekende hoofdstukken:
  - Richtlijnen voor keuze en aanschaf
  - Evaluatie en onderhoud van beleid
Ontbrekende paragrafen:
  - Richtlijnen voor keuze en aanschaf > Selectiekaders
  - Richtlijnen voor keuze en aanschaf > Aanschafproces
  - Evaluatie en onderhoud van beleid > Frequentie en proces

==== Concept Leermiddelenbeleid Penta Jacob van Liesveldt ====
Hoofdstukscore: 100.0%
Paragraafscore: 92.9%
Alle hoofdstukken aanwezig!
Ontbrekende paragrafen:
  - Evaluatie en onderhoud van beleid > Frequentie en proces


<a name='4.2'></a>
<h3>&sect;4.2: Volledigheidscheck </h3>

Om te beoordelen of alle vragen beantwoord waren in de tekst heb ik gebruik gemaakt van Gemini 2.5 Flash. Ik heb deze alle paragrafen laten beoordelen op de aanwezigheid van de antwoorden op de vragen, en hier voor iedere paragraaf een score aan laten geven. Deze resultaten zijn per school terug te vinden in deze paragraaf. De chat staat [hier](https://gemini.google.com/app/5b651a2372029041?hl=nl).

<a name='4.2.1'></a>
<h3>&sect;4.2.1: Volledigheidscheck Penta Jacob van Liesveldt College</h3>

| Paragraaf                               | Score | Max. Score |
| :-------------------------------------- | :---- | :--------- |
| 1.1 Aanleiding                          | 2     | 2          |
| 1.2 Reikwijdte                          | 2     | 2          |
| 1.3 Doel                                | 2     | 2          |
| 2.1 Kernwaarden en onderwijsdoelen      | 2     | 2          |
| 2.2 Ondersteuning door leermiddelen     | 2     | 2          |
| 3.1 ICT                                 | 1     | 2          |
| 3.2 Financiën                           | 2     | 2          |
| 3.3 HR – Deskundigheid en ondersteuning | 2     | 2          |
| 4.1 Selectiekaders                      | 2     | 2          |
| 4.2 Aanschafproces                      | 2     | 2          |
| 5.1 Leermiddelenmix                     | 2     | 2          |
| 5.2 Praktijkgebruik                     | 2     | 2          |
| 6.1 Gebruik (Leermiddelenbeleid onderhouden) | 0     | 2          |
| 6.2 Frequentie en proces                 | 2     | 2          |
| **Totaal** | **25**| **28** |

In [25]:
print(f"Totaalpercentage beantwoord: {(25 / 28) * 100:.2f}%") 

Totaalpercentage beantwoord: 89.29%


<a name='4.2.2'></a>
<h3>&sect;4.2.2: Volledigheidscheck Johannes Calvijn</h3>

| Paragraaf                               | Score | Max. Score |
| :-------------------------------------- | :---- | :--------- |
| 1.1 Aanleiding                          | 2     | 2          |
| 1.2 Reikwijdte                          | 2     | 2          |
| 1.3 Doel                                | 2     | 2          |
| 2.1 Kernwaarden en onderwijsdoelen      | 2     | 2          |
| 2.2 Ondersteuning door leermiddelen     | 1     | 2          |
| 3.1 ICT                                 | 2     | 2          |
| 3.2 Financiën                           | 2     | 2          |
| 3.3 HR – Deskundigheid en ondersteuning | 1     | 2          |
| 4.1 Selectiekaders                      | 2     | 2          |
| 4.2 Aanschafproces                      | 2     | 2          |
| 5.1 Samenstelling van de leermiddelenmix | 2     | 2          |
| 5.2 Praktijkgebruik en afstemming       | 0     | 2          |
| 6.1 Gebruik (Leermiddelenbeleid onderhouden) | 0     | 2          |
| 6.2 Frequentie en proces                 | 2     | 2          |
| **Totaal** | **22**| **28** |

In [24]:
print(f"Totaalpercentage beantwoord: {(22 / 28) * 100:.2f}%") 

Totaalpercentage beantwoord: 78.57%


<a name='4.2.3'></a>
<h3>&sect;4.2.3: Volledigheidscheck Liemers School</h3>

| Paragraaf                               | Score | Max. Score |
| :-------------------------------------- | :---- | :--------- |
| 1.1 Aanleiding                          | 2     | 2          |
| 1.2 Reikwijdte                          | 2     | 2          |
| 1.3 Doel                                | 2     | 2          |
| 2.1 Kernwaarden en onderwijsdoelen      | 2     | 2          |
| 2.2 Ondersteuning door leermiddelen     | 2     | 2          |
| 3.1 ICT-infrastructuur en digitale randvoorwaarden | 1     | 2          |
| 3.2 Financiële kaders en budgettering   | 1     | 2          |
| 3.3 Personele randvoorwaarden           | 1     | 2          |
| 4.1 Keuzeproces en verantwoordelijkheden | 1     | 2          |
| 4.2 Inkoop, aanbesteding en leverancierskeuze | 2     | 2          |
| 5.1 Mix van leermiddelen                | 2     | 2          |
| 5.2 Praktijkgebruik en didactische inzet| 0     | 2          |
| 6.1 Evaluatie en actualisatie           | 0     | 2          |
| 6.2 Frequentie en proces                | 2     | 2          |
| **Totaal** | **20**| **28** |

In [18]:
print(f"Totaalpercentage beantwoord: {(20 / 28) * 100:.2f}%") 

Totaalpercentage beantwoord: 71.43%


<a name='4.2.4'></a>
<h3>&sect;4.2.4: Volledigheidscheck Penta Jacob van Liesveldt College OpenAI API</h3>

| Paragraaf                               | Score | Max. Score |
| :-------------------------------------- | :---- | :--------- |
| 1.1 Aanleiding                          | 2     | 2          |
| 1.2 Reikwijdte                          | 2     | 2          |
| 1.3 Doel                                | 2     | 2          |
| 2.1 Kernwaarden en onderwijsdoelen      | 2     | 2          |
| 2.2 Ondersteuning door leermiddelen     | 2     | 2          |
| 3.1 ICT                                 | 1     | 2          |
| 3.2 Financiën                           | 2     | 2          |
| 3.3 HR – Deskundigheid en ondersteuning | 2     | 2          |
| 4.1 Selectiekaders                      | 2     | 2          |
| 4.2 Aanschafproces                      | 2     | 2          |
| 5.1 Leermiddelenmix                     | 2     | 2          |
| 5.2 Praktijkgebruik                     | 1     | 2          |
| 6.1 Gebruik                             | 1     | 2          |
| 6.2 Frequentie en Proces                | 2     | 2          |
| **Totaal** | **25**| **28** |

In [19]:
print(f"Totaalpercentage beantwoord: {(25 / 28) * 100:.2f}%") 

Totaalpercentage beantwoord: 89.29%


<a name='4.3'></a>
<h3>&sect;4.3: Bruikbaarheid </h3>

Voor de controle op de bruikbaarheid van de gegenereerde leermiddelenbeleidsplannen hebben wij enkele van onze teksten laten beoordelen door onze odprachtgever. Dit was een mail over de gegenereerde versie van een concept leermiddelenbeleid van de Johannes Calvijnschool in Urk. Hiervoor hadden wij de gegenereerde versie van het Penta Jacob van Liesveldt College laten beoordelen en hebben wij feedback gekregen die wij hebben verwerkt voor de tweede tekst met de school uit Urk.

![Mail opdrachtgever](mail-anneke.png)

<a name='4.4'></a>
<h3>&sect;4.4: Objectieve kwaliteitsmetingen </h3>

Om de scores goed te kunnen vergelijken met de al bestaande leermiddelenbeleidsplannen worden hier onder de statistieken weergegeven die terug kwamen uit deze bestaande bestanden. Hiermee kan er vergeleken worden of de resultaten van de gegenereerde beleidsplannen niet te erg afwijken.

![Gemiddelde scores andere scholen](gemiddelde_scores.png)

In [13]:
stats = []
for school, text in school_sections.items():
    tokens = word_tokenize(text)
    word_lengths = [len(word) for word in tokens]
    avg_word_length = sum(word_lengths) / len(word_lengths) if tokens else 0
    vocab_diversity = len(set(tokens)) / len(tokens) if tokens else 0
    flesch_score = textstat.flesch_reading_ease(text)
    stats.append({
        'School': school,
        'Average Word Length': avg_word_length,
        'Total Words': len(tokens),
        'Vocabulary Diversity': vocab_diversity,
        'Flesch Reading Ease': flesch_score
    })

df_stats = pd.DataFrame(stats)
display(df_stats)

,School,Average Word Length,Total Words,Vocabulary Diversity,Flesch Reading Ease
0,(OpenAI API) Concept LMB Jacob van liesveldt,5.497980,4456,0.259201,35.98
1,Concept Leermiddelenbeleid Johannes Calvijnschool,5.604392,3142,0.287397,36.69
2,Concept Leermiddelenbeleid Liemers College,6.058585,5428,0.284451,20.18
3,Concept Leermiddelenbeleid Penta Jacob van Lie...,5.580054,3329,0.304296,38.62


<a name='4.5'></a>
<h3>&sect;4.5: Ethische aspecten </h3>

In [14]:
ethiek_criteria = {
    "Belanghebbenden genoemd": [
        "leerling", "student", "docent", "leerkracht", "ouder", "ouders", "team", "bestuur", "stakeholder"
    ],
    "Transparantie over verantwoordelijkheid": [
        "verantwoordelijk", "verantwoordelijkheid", "besluitvorming", "proces", "transparant", "verantwoording"
    ],
    "Privacy en informatieveiligheid": [
        "privacy", "gegevensbescherming", "avg", "veiligheid", "informatiebeveiliging", "data protection"
    ],
    "Inclusiviteit en gelijkwaardigheid": [
        "inclusief", "gelijkheid", "gelijke kansen", "diversiteit", "toegankelijk", "inclusiviteit", "discriminatie"
    ],
    "Mogelijkheid tot feedback of bezwaar": [
        "feedback", "klacht", "bezwaar", "inspraak", "reactie", "evaluatie", "aanpassing", "melding"
    ]
}

In [15]:
ethiek_stats = []
for school, text in school_sections.items():
    text_lower = text.lower()
    school_result = {"School": school}
    for criterium, keywords in ethiek_criteria.items():
        gevonden = any(word in text_lower for word in keywords)
        school_result[criterium] = gevonden
    ethiek_stats.append(school_result)

df_ethiek = pd.DataFrame(ethiek_stats)
display(df_ethiek)

,School,Belanghebbenden genoemd,Transparantie over verantwoordelijkheid,Privacy en informatieveiligheid,Inclusiviteit en gelijkwaardigheid,Mogelijkheid tot feedback of bezwaar
0,(OpenAI API) Concept LMB Jacob van liesveldt,True,True,True,True,True
1,Concept Leermiddelenbeleid Johannes Calvijnschool,True,True,True,True,True
2,Concept Leermiddelenbeleid Liemers College,True,True,True,True,True
3,Concept Leermiddelenbeleid Penta Jacob van Lie...,True,True,True,True,True


In [16]:
for school, text in school_sections.items():
    print(f"\n=== {school} ===")
    text_lower = text.lower()
    for criterium, keywords in ethiek_criteria.items():
        print(f"\nCriterium: {criterium}")
        hits = [word for word in keywords if word in text_lower]
        if hits:
            print(f"Gevonden signalen: {', '.join(hits)}")
            for hit in hits:
                start = text_lower.find(hit)
                context = text[max(0, start-40):start+40]
        else:
            print("Geen relevante woorden gevonden.")


=== (OpenAI API) Concept LMB Jacob van liesveldt ===

Criterium: Belanghebbenden genoemd
Gevonden signalen: leerling, student, docent, ouder, ouders, team

Criterium: Transparantie over verantwoordelijkheid
Gevonden signalen: verantwoordelijk, verantwoordelijkheid, besluitvorming, proces, transparant, verantwoording

Criterium: Privacy en informatieveiligheid
Gevonden signalen: privacy, gegevensbescherming, avg, veiligheid

Criterium: Inclusiviteit en gelijkwaardigheid
Gevonden signalen: inclusief, gelijkheid, diversiteit, toegankelijk

Criterium: Mogelijkheid tot feedback of bezwaar
Gevonden signalen: feedback, evaluatie, aanpassing

=== Concept Leermiddelenbeleid Johannes Calvijnschool ===

Criterium: Belanghebbenden genoemd
Gevonden signalen: leerling, leerkracht, ouder, ouders, team, bestuur

Criterium: Transparantie over verantwoordelijkheid
Gevonden signalen: verantwoordelijk, verantwoordelijkheid, besluitvorming, proces, transparant

Criterium: Privacy en informatieveiligheid
G

Alle gegenereerde leermiddelenbeleidsplannen komen door de ethische check heen, kijkend alleen naar woorden die voor moeten komen in de teksten. Het is wel belangrijk om een uitgebreidere ethische check uit te voeren bij het implementeren van een aanpak in de toekomst.

[Back to the top](#start)
<a name='5.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H5: Conclusie</strong></h2>
</div>

<a name='5.1'></a>
<h3>&sect;5.1: Custom GPT </h3>

- ***Johannes Calvijnschool (Gegenereerd met GPT Plus):*** Dit beleid is qua structuur bijna volledig, met alle hoofdstukken en ongeveer 93% van de paragrafen aanwezig. Alleen "Frequentie en proces" ontbreekt. 78.6% van de voorbeeldvragen is beantwoord. De tekst is redelijk toegankelijk en alle etische criteria zijn benoemd in het document.
- ***Liemers College (Gegenereerd met gratis GPT):*** Dit beleid scoort het laagst op structuur: 66.7% van de hoofdstukken en 78.6% van de paragrafen is aanwezig, met belangrijke ontbrekende onderdelen. Ook zijn de minste voorbeeldvragen beantwoord (71.4&). De tekst is lang en moeilijk leesbaar, maar ethisch gezien wel volledig.
- ***Penta Jacob van Liesveldt (Gegenereerd met GPT Plus):*** Deze school scoort samen met de Johannes Calvijnschool het hoogst op structuur: alle hoofdstukken, ongeveer 93% van de paragrafen (ook hier ontbreekt alleen "Frequentie en proces"). Het meeste aantal voorbeeldvragen is beantwoord (89.3%). De tekst heeft de grootste vocabulaire-diversiteit en is het beste leesbaar. Alle ethische criteria zijn genoemd.

##### Algemene oordeel:

Uit de evaluatie blijkt dat de beleidsdocumenten die met een GPT Plus-account zijn gegenereerd (Penta Jacob van Liesveldt en Johannes Calvijnschool) over het algemeen beter scoren op structuur, volledigheid en leesbaarheid dan het document dat met een gratis GPT-account is gemaakt (Liemers College). 

Het beleid van Penta Jacob van Liesveldt springt er in het bijzonder uit als best presterende document. Dit suggereerd dat het gebruik van GPT Plus kan bijdragen aan een hogere kwaliteit en volledigheid van gegenereerde beleidsdocumenten, al blijven inhoudelijke controle en nabewerking door mensen altijd belangrijk.

<a name='5.2'></a>
<h3>&sect;5.2: OpenAI API </h3>

Het met de OpenAI API gemaakte leermiddelenbeleid voor het Jacob van Liesveldt is volledig qua structuur: alle hoofdstukken en paragragen zijn aanwezig. Ook qua beantwoording van voorbeeldvragen scoort het document hoog (89.3%). De tekst is helder opgebouwd, compact en goed leesbaar, al is de vocabulaire-diversiteit iets lager dan bij de custom GPT-aanpak. Alle ethische criteria worden duidelijk benoemd.

<a name='5.3'></a>
<h3>&sect;5.3: Vergelijking </h3>

Uit de resultaten blijkt dat beide aanpakken tot een zeer compleet leermiddelenbeleid kunnen leiden. De OpenAI API- aanpak springt eruit op structuur: het document bevat als enige zowel alle hoofdstukken als alle paragrafen volledig, zonder ontbrekende onderdelen. Qua volledigheid van beantwoorden van voorbeeldvragen scoren de OpenAI-API en de beste custom-GPT aanpak gelijk (beide 89.3%).

Op het gebied van objectieve tekstkwaliteit zien we kleine verschillen:
-  De custom GPT-aanpak levert een iets hogere vocabulaire-diversiteit op, wat kan wijzen op meer variatie in woordgebruik.
- De OpenAI API-tekst is iets compacter qua woordlengte en het totaal aantal woorden, met vergelijkbare leesbaarheid.

Wat betreft ethische criteria voldoen beide aanpakken volledig, met lichte nuance verschillen en benoemde belanghebbende en details.

##### Samengevat:
De OpenAI API-aanpak levert het meest complete leermiddelenbeleid op het gebied van structuur, terwijl de custom GPT-aanpak nét wat meer variantie in taalgebruik biedt. Beide methoden zijn in staat om kwalitatief sterke, ethisch verantwoorde documenten te genereren, maar voor maximale volledigheid van structuur scoort de OpenAI API heft hoogste, namelijk compleet aanwezig.

[Back to the top](#start)
<a name='6.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H6: Bijlagen</strong></h2>
</div>

<a name='6.1'></a>
<h3>&sect;6.1: Checklist Structuuranalyse </h3>

Onderstaande checklist is gebruikt om te kijken of alle paragrafen en hoofdstukken terugkomen in het gegenereerde leermiddelenbeleid:

- [ ] Aanleiding, reikwijdte en doel
    - [ ] Aanleiding
    - [ ] Reikwijdte
    - [ ] Doel
- [ ] Onderwijskundige visie en doel
    - [ ] Kernwaarden en onderwijsdoelen
    - [ ] Ondersteuning door leermiddelen
- [ ] Randvoorwaarden
    - [ ] ICT
    - [ ] Financiën
    - [ ] HR
- [ ] Richtlijnen voor keuze en aanschaf leermiddelen
    - [ ] Selectiekaders
    - [ ] Aanschafproces
- [ ] Leermiddelenmix en praktijkgebruik
    - [ ] Leermiddelenmix
    - [ ] Praktijkgebruik
- [ ] Evaluatie en onderhoud van beleid
    - [ ] Gebruik
    - [ ] Frequenie en proces

<a name='6.2'></a>
<h3>&sect;6.2: Vragenlijst invulling </h3>

De vragenlijst hieronder is gebruikt om te beoordelen of alle vragen beantwoord zijn in het gegenereerde leermiddelenbeleid voor iedere paragraaf.

- [ ] Aanleiding, reikwijdte en doel 
    - [ ] Aanleiding  
        - [ ] Wat is de directe aanleiding voor het opstellen van een (nieuw) leermiddelenbeleid? Bijvoorbeeld strategische doelen, evaluatie huidige middelen, opbrengsten (interne factoren). 
        - [ ] Zijn er wettelijke of aanbestedingseisen die om een (vernieuwd) leermiddelenbeleid vragen? Bijvoorbeeld aanbestedingsplicht, nieuwe onderwijstrends of veranderd curriculum (externe factoren). 
    - [ ] Reikwijdte 
        - [ ] Op welke vakken/leergebieden is het leermiddelenbeleid van toepassing? 
        - [ ] Wat verstaan we onder leermiddelen? (Over welke leermiddelen doet het leermiddelenbeleid een uitspraak?) 
        - [ ] Voor wie is het leermiddelenbeleid bedoeld? 
    - [ ] Doel 
        - [ ] Wat zijn de belangrijkste doelen die we met dit beleid willen bereiken? Bijvoorbeeld bewuste keuzes rondom leermiddelen, integratie van digitale middelen, gepersonaliseerd leren of duurzaamheid. 
- [ ] Onderwijskundige visie en doel 
    - [ ] Wat zijn de kernwaarden en onderwijsdoelen van onze organisatie? 
    - [ ] Wat zijn de onderwijsdoelen van de verschillende scholen binnen onze organisatie?  
    - [ ] Hoe kunnen leermiddelen deze kernwaarden en onderwijsdoelen ondersteunen? 
    - [ ] Welke specifieke onderwijsdoelen willen we via leermiddelen realiseren? Bijvoorbeeld vakoverstijgend werken of formatief onderwijs. 
- [ ] Randvoorwaarden 
    - [ ] ICT 
        - [ ] Wat zijn de ICT-vereisten voor leermiddelen? Bijvoorbeeld ten aanzien van infrastructuur, technische eisen, toegankelijkheid en interoperabiliteit. 
        - [ ] Hoe gaan we om met privacy- en informatieveiligheid bij digitale leermiddelen? 
        - [ ] Wat is ons standpunt ten aanzien van onderliggende technologieën in digitale leermiddelen? 
    - [ ] Financiën 
        - [ ] Wat is het budget voor leermiddelen en wat is de bestedingsruimte per leerling? 
        - [ ] Wordt het budget in een bepaalde verhouding verdeeld over verschillende soorten leermiddelen, vakken of leerjaren? Hoe? 
        - [ ] Wat is de mate van vrijheid voor scholen uit het bestuur om het budget in te vullen? 
        - [ ] Op welke manier moeten gemaakte keuzes onderbouwd worden om de investering te rechtvaardigen? 
    - [ ] HR 
        - [ ] Welke vaardigheden en competenties hebben leraren nodig om de leermiddelen effectief in te zetten? Bijvoorbeeld digitale didactiek, lesmateriaal arrangeren, differentiëren, toetsconstructie, projectmatig werken. 
        - [ ] Hoe willen we leraren ondersteunen bij het gebruik van nieuwe leermiddelen? 
        - [ ] Wat is de professionaliseringsbehoefte van stafmedewerkers om teams en processen rondom leermiddelen te ondersteunen? 
        - [ ] Welke professionaliseringsthema's stellen we centraal?  
        - [ ] Hoe bieden we professionalisering aan? Bijvoorbeeld via gezamenlijke studiedagen, (online) trainingen of persoonlijke opleidingstrajecten. 
- [ ] Richtlijnen voor keuze en aanschaf van leermiddelen 
    - [ ] Selectiekaders 
        - [ ] Zie randvoorwaarden bij 3. 
        - [ ] Wat is de vervangingscyclus en afschrijvingstermijn van folio en digitale leermiddelen? 
        - [ ] Welke afspraken zijn er voor contracttermijnen (contractcyclus)? 
        - [ ] Welke overige aspecten vinden we van belang voor keuze en aanschaf van leermiddelen? Bijvoorbeeld duurzaamheid. 
    - [ ] Aanschafproces 
        - [ ] Hoe ziet het proces voor aanschaf van leermiddelen eruit, van selectie tot implementatie, in rollen en verantwoordelijkheden? 
        - [ ] Hoe ziet het proces voor aanschaf van leermiddelen eruit, van selectie tot implementatie, in logistieke en administratieve afhandeling? 
- [ ] Leermiddelenmix en Praktijkmix
    - [ ] Leermiddelenmix 
        - [ ] Welke soorten leermiddelen (folio, digitaal, open/gesloten) dragen bij aan het realiseren van onze onderwijsdoelen? 
        - [ ] Wat is de gewenste verhouding tussen folio en digitale en tussen open en gesloten leermiddelen binnen onze leermiddelenmix?  
    - [ ] Praktijkmix
        - [ ] Hoe monitoren we het daadwerkelijke gebruik van leermiddelen in de klas? Bijvoorbeeld door overzichten van daadwerkelijk ingezette leermiddelen en gebruiksanalyses van digitale leermiddelen. 
        - [ ] Hoe evalueren we de effectiviteit en het gebruik van leermiddelen? Bijvoorbeeld door monitoren van opbrengsten. 
        - [ ] Wie zijn er betrokken bij de evaluatie van de leermiddelen? Bijvoorbeeld lerarenteams, leermiddelencoördinator en leerlingen. 
- [ ] Evaluatie en onderhoud van het beleid
    - [ ] Gebruik 
        - [ ] In welke situaties en bij welke besluiten moet het leermiddelenbeleid geraadpleegd worden? Bijvoorbeeld aan het begin van elk leermiddelenkeuzeproces en bij de verantwoording van gemaakte keuzes. 
    - [ ] Frequentie en proces 
        - [ ] Hoe vaak evalueren we het leermiddelenbeleid? 
        - [ ] Naar aanleiding waarvan evalueren we het leermiddelenbeleid? Bijvoorbeeld bij veranderingen in het curriculum, onderwijstrends, technologische ontwikkelingen, veranderingen in opbrengsten. 
        - [ ] Hoe evalueren we het leermiddelenbeleid? Bijvoorbeeld met feedbackrondes en overleg met betrokken onderwijsprofessionals. 

<a name='6.3'></a>
<h3>&sect;6.3: Checklist Ethische aspecten </h3>

Van de onderstaande lijst moet er voor ieder onderwerp minimaal 1 van de opgegeven woorden voorkomen om te voldoen aan de ethische check.


- [ ] Belanghebbenden genoemd
    - [ ] leerling
    - [ ] student
    - [ ] docent
    - [ ] leerkracht
    - [ ] ouder
    - [ ] ouders
    - [ ] team
    - [ ] bestuur
    - [ ] stakeholder
- [ ] Transparantie over verantwoordelijkheid
    - [ ] verantwoordelijk
    - [ ] verantwoordelijkheid
    - [ ] besluitvorming
    - [ ] proces
    - [ ] transparant
    - [ ] verantwoording
- [ ] Privacy en informatieveiligheid
    - [ ] privacy
    - [ ] gegevensbescherming
    - [ ] avg
    - [ ] veiligheid
    - [ ] informatiebeveiliging
    - [ ] data protection
- [ ] Inclusiviteit en gelijkwaardigheid
    - [ ] inclusief
    - [ ] gelijkheid
    - [ ] gelijke kansen
    - [ ] diversiteit
    - [ ] toegankelijk
    - [ ] inclusiviteit
    - [ ] discriminatie
- [ ] Mogelijkheid tot feedback of bezwaar
    - [ ] feedback
    - [ ] klacht
    - [ ] bezwaar
    - [ ] inspraak
    - [ ] reactie
    - [ ] evaluatie
    - [ ] aanpassing
    - [ ] melding

[Back to the top](#start)
<a name='7.0'></a>

<div style="background-color:#783465; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>Literatuurlijst</strong></h2>
</div>

- *Evaluatietechnieken.* (2025). Datalab V https://dehaagsehogeschool-my.sharepoint.com/:w:/r/personal/22162038_student_hhs_nl/_layouts/15/Doc.aspx?sourcedoc=%7B9F01B2FA-2E69-4F9D-9FFE-00FB1BBBE2D2%7D&file=Evaluatietechnieken.docx&action=default&mobileredirect=true